## Week 6 workshop

In this week, we'll continue to explore the GPT-style model trained on Shakespeare text.

First we import the required dependencies:

In [ ]:
import sys
import torch
import polars as pl
#from plotnine import ggplot, aes, geom_line, labs, theme_minimal

sys.path.append("scratch-llm")
from model.llm import LLM
from model.tokenizer import Tokenizer
from helpers.config import LLMConfig, get_device

Next we prepare the model setup, model, and tokenizer.

In [ ]:
# the model setup (has to be consistent with the trained model we're loading)
llm_config = LLMConfig(
    vocab_size = 4096,
    seq_len = 128,
    dim_emb = 256,
    num_layers = 4,
    num_heads = 8,
    emb_dropout = 0.0,
    ffn_dim_hidden = 4 * 256,
    ffn_bias = False
)

# the trained tokenizer
tokenizer = Tokenizer("data/tinyshakespeare.model")

# the model object
model = LLM(
    vocab_size = tokenizer.vocab_size,
    seq_len = llm_config.seq_len,
    dim_emb = llm_config.dim_emb,
    num_layers = llm_config.num_layers,
    attn_num_heads = llm_config.num_heads,
    emb_dropout = llm_config.emb_dropout,
    ffn_hidden_dim = llm_config.ffn_dim_hidden,
    ffn_bias = llm_config.ffn_bias
)

# the device on which we're running this (CPU vs GPU etc.)
device = get_device()

# move the model to the appropriate GPU device
model.to(device)

# load the saved model weights
model.load_state_dict(torch.load(
    "data/tinyshakespeare_llm.pt",
    weights_only = True,
    map_location = device
))

# put the model into evaluation mode
model.eval()

## Exploring the tokenizer

Extract all tokens and print out two specific ones.

In [ ]:
# extract list of all tokens
tokens = [tokenizer.sp.id_to_piece(i) for i in range(llm_config.vocab_size)]

print(tokenizer.sp.piece_to_id("▁perforce"))
print(tokenizer.sp.piece_to_id("▁basilisk"))

Print all tokens the tokenizer knows.  The underscore ("▁") in front of a token indicates the beginning of a word.

In [ ]:
tokens_per_line = 10
for i in range(0, len(tokens), tokens_per_line):
    line_tokens = tokens[i:i+tokens_per_line]
    print(f"[{i:5d}-{min(i+tokens_per_line-1, len(tokens)-1):5d}] {' | '.join(line_tokens)}")

## Exploring model parameters

Structure of the model parameters:

- **0** :: weight matrix for **token embeddings**

<!-- -->

- **1** :: **RMSNorm** parameter vector
- **2** :: Q, K, V matrices (concatenated) for **MultiHeadAttention**
- **3** :: weight matrix for projout part of **MultiHeadAttention**
- **4** :: **RMSNorm** parameter vector
- **5** :: initial weight matrix for **FeedForward (SwiGLU)** part
- **6** :: **SwiGLU** weight matrices (concatened)
- **7** :: **SwiGLU** bias vector
- **8** :: final weight matrix for **FeedForward (SwiGLU)** part

<!-- -->

-  **9-16** :: as 1-8 but for second TransformerBlock
- **17-24** :: third TransformerBlock
- **25-32** :: fourth TransformerBlock

<!-- -->

- **33** :: **RMSNorm** parameter vector
- **34** :: final **projection_head** bias vector

**NOTE**: there is no weight matrix for the final projection head b/c it is "weight-tied" to the token embeddings weight matrix (0 above)

In [ ]:
# extract all model parameters
parList = list(model.parameters())

# extract tensor shapes for each tensor
parShapes = [list(el.shape) for el in parList]
parShapes

Extracting token embeddings. These are the initial embeddings going into the transformer module.

In [ ]:
# extract list of all tokens (just as before)
tokens = [tokenizer.sp.id_to_piece(i) for i in range(llm_config.vocab_size)]

# obtain tensor of token embeddings and convert numpy array for downstream manipulation
embedding_data = parList[0].cpu().detach()
print(embedding_data.shape) # (4096, 256)

# _basilisk is token 4077
print(embedding_data[4077, :])


In [ ]:
# create DataFrame with token column plus embedding dimensions
embeddings = pl.DataFrame({
    'token': tokens,
    'token_id': [i for i in range(llm_config.vocab_size)],
    **{f'dim_{i}': embedding_data[:, i] for i in range(embedding_data.shape[1])}
})

# print token embeddings for three chosen tokens
target_tokens = ["▁basilisk", "▁perforce", "▁castle"]
embeddings.filter(pl.col('token').is_in(target_tokens))

We can extract embeddings from each layer using hooks. Hooks are call-back functions that get called when a particular part of the model is executed, and so they allow us to capture input and output of those parts of the model.

In [ ]:
# set up a list to store embeddings from each transformer layer
layer_outputs = []

# hook function that stores the output of a model component
def hook_fn(module, input, output):
    layer_outputs.append(output.detach().clone().cpu())

# register hooks on each transformer block
hooks = []
for block in model.transformer:
    hook = block.register_forward_hook(hook_fn)
    hooks.append(hook) # we need to keep a list of all hooks so we can remove them at the end

# run forward pass
prompt = tokenizer.encode(
    "I say, basilisk, perforce, castle.",
    beg_of_string = True,
    pad_seq = True,
    seq_len = llm_config.seq_len
)
inputs = torch.tensor(prompt, dtype=torch.int32).unsqueeze(0)

# print the input tensor
print(f"The prompt input:\n{inputs}\n")

# generate output
# we don't actually need the output, we just do this to call the `forward()` function of the model
out = model(inputs.to(device))

# clean up hooks (so we can run the cell again in the same session and not accumulate hooks)
for hook in hooks:
    hook.remove()

The tokens for basilisk, perforce, and castle are at positions 122, 124, and 126 in this tensor.

In [ ]:
token_pos = [122, 124, 126]
inputs[0, token_pos]

These are the corresponding embeddings in layer 3 (the final layer). We will have to do a bit more work to look at them easily.

In [ ]:
print(layer_outputs[3][0, token_pos, :].shape)
layer_outputs[3][0, token_pos, :]

In [ ]:
# function to create data frame of embedding data for given layer
def make_embedding_table(layer_outputs, layer_id):
    df = pl.DataFrame({
        'token': [tokenizer.sp.id_to_piece(inputs[0, i].item()) for i in range(llm_config.seq_len)],
        'token_id': inputs[0, :],
        **{f'dim_{i}': layer_outputs[layer_id][0, :, i] for i in range(llm_config.dim_emb)}
    })
    return df

print("All layer 0 embeddings:")
make_embedding_table(layer_outputs, 0)[token_pos]


Now we print out the embedding tables only for the target tokens, plus once more the initial token embeddings for reference.

In [ ]:
embeddings.filter(pl.col('token').is_in(target_tokens)) # input embeddings

In [ ]:
make_embedding_table(layer_outputs, 0)[token_pos] # embeddings after layer 0

In [ ]:
make_embedding_table(layer_outputs, 1)[token_pos] # embeddings after layer 1

In [ ]:
make_embedding_table(layer_outputs, 2)[token_pos] # embeddings after layer 2

In [ ]:
make_embedding_table(layer_outputs, 3)[token_pos] # embeddings after layer 3